# Compare GM outputs from notebook and odc-stats

In [ ]:
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray
import os
from odc.stats.plugins.gm import StatsGMS2
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct
from odc.geo.geobox import GeoBox

# Quick fix to affine object issue
import patch_odc_stats

### 1. Create Tasks

In [ ]:
# Function to save task with odc-stats
def save_tasks(cloud_cover, output_db):
    dataset_filter = json.dumps({"cloud_cover": [0, cloud_cover]})
    
    !uv run odc-stats save-tasks \
        --frequency "annual" \
        --grid "EPSG:6933;10;5000" \
        --year "2024" \
        --input-products "s2_l2a" \
        --dataset-filter='{{"cloud_cover": [0,{cloud_cover}]}}' \
        {output_db}

In [ ]:
# Execute save_tasks
save_tasks(60, "s2_l2a_2024--P1Y.db")

### 2. Explore Task

In [ ]:
# Load task
grid = gpd.read_file("s2_l2a_2024--P1Y-2024--P1Y.geojson")

In [ ]:
# Show task interactively, used to pick test tile
grid.explore(
    color="blue",
    style_kwds={"fillOpacity": 0.2, "weight": 0.5, "color": "blue"}
)

## Accessing task database

In [ ]:
# define x, y index and year, which has to be aligned with what's used in odc-stats save-tasks

x, y = 235, -10
year = "2024--P1Y"
cloud_filters= {"cloud shadows":[["dilation", 5]], "cloud medium probability":[["opening", 5], ["dilation", 5]], "cloud high probability":[["opening", 5], ["dilation", 5]], "thin cirrus":[["dilation", 5]]}

In [ ]:
# Dummy product to pass to taskreader

name, version = 'gm_s2', '0-0-1' # product name and version (appended to results path)
product = OutputProduct(name=name,
                        version=version,
                        short_name=name,
                        location = "",
                        properties = {},
                        measurements = [],
                       )

tidx = (year, x, y)
rdr = TaskReader("s2_l2a_2024--P1Y.db", product=product)
task = rdr.load_task(tidx)
task

## Compare datasets in Task to datasets within Database

In [ ]:
import datacube

# Connect to ODC
dc = datacube.Datacube(app="geomad_s2")

In [ ]:
sentinel2_datasets = dc.find_datasets(
    product=["s2_l2a"],
    time="2024",
    like=task.geobox.to_crs("EPSG:6933"),
    cloud_cover=(0, 60),
)

In [ ]:
s2_water_filtered = dc.find_datasets(
    product=["s2_l2a"],
    time="2024",
    like=task.geobox.to_crs("EPSG:6933"),
    cloud_cover=(0, 60),
    s2_water_percentage=(0, 50)
)

In [ ]:
# expect same number of datasets found
len(task.datasets), len(sentinel2_datasets), len(s2_water_filtered)

## Visualise Tile

In [ ]:
# Load available data
ds = dc.load(
    datasets=sentinel2_datasets,
    geopolygon=task.geobox.extent,  
    measurements=["red", "green", "blue", "scl"],
    dask_chunks={"x": 5000, "y": 5000, "time":-1},
    resolution=(-10,10),
    group_by="solar_day",
    output_crs=task.geobox.crs, #"EPSG:6933"
    driver="rio",
)

In [ ]:
# expect same number of days for this tile in s2_l2a_2024--P1Y.csv
ds

In [ ]:
task.geobox.explore()

## Process Data with ODC Stats Run

In [ ]:
tile_id = f"{year}/{x:+03d}/{y:+03d}"
output_dir = os.path.abspath("./output")
file_url_output = f"file://{output_dir}"

In [ ]:
!uv run odc-stats run \
    --plugin="gm-s2" \
    --plugin-config="../config/geomad_config/plugin_config.yaml" \
    --config="../config/geomad_config/main_config.yaml" \
    --threads=3 \
    --memory-limit="29Gi" \
    --location="{file_url_output}" \
    s2_l2a_2024--P1Y.db \
    {tile_id}

## Visualise GeoMAD output

In [ ]:
output_dir = "/home/jovyan/Indonesia-GeoMAD/notebook/output/x235/y-10/2024--P1Y"
tile_name = "gm_s2_annual_x235y-10_2024--P1Y"

rgba = rioxarray.open_rasterio(f"{output_dir}/{tile_name}_rgba.tif")

rgba.plot.imshow(rgb='band', figsize=(6, 6), robust=True)
plt.title("GeoMAD RGB")
plt.show()

## Process Data with ODC Algo

In [ ]:
ds["scl"].attrs['flags_definition'] =dc.list_measurements().loc['s2_l2a','scl']['flags_definition']
ds["scl"].attrs['flags_definition']

In [ ]:
from odc.algo import enum_to_bool, erase_bad
from odc.algo import mask_cleanup

# Apply the same cloud filter
# Erase Data Pixels for which mask == nodata
mask = ds["scl"]
bad = enum_to_bool(mask, ("no data",))
for cloud_class, c_filter in cloud_filters.items():
    if not isinstance(cloud_class, tuple):
        cloud_class = (cloud_class,)
        cloud_mask = enum_to_bool(mask, cloud_class)
        cloud_mask_buffered = mask_cleanup(cloud_mask, mask_filters=c_filter)
        bad = cloud_mask_buffered | bad

ds = ds.drop_vars(["scl"])
ds = erase_bad(ds, bad)

In [ ]:
from odc.algo import geomedian_with_mads
gm = geomedian_with_mads(ds)

In [ ]:
gm

### Option - use GM plugin to process

In [ ]:
# Use gm plugin to generate GM
# use same configuration as 
gm = StatsGMS2(resampling="nearest", 
               bands=["blue", "green", "red", "nir", "swir16", "swir22"], 
                cloud_filters=cloud_filters,
               work_chunks=(100,100),
)
xx = gm.input_data(task.datasets, task.geobox)
xx = gm.native_transform(xx)
output = gm.reduce(xx)

In [ ]:
output

In [ ]:
#compute
#output = output.compute()

### Load odc-stats run output

In [ ]:
import xarray as xr

#Assume output save in current location
basepath = f"x{x:03d}/y{y:03d}/{year}/gm_s2_x{x:03d}y{y:03d}_{year}"

bands = ['red', 'green', 'blue']
datasets = [xr.open_dataset(f'{basepath}_{band}.tif',chunks ={}).squeeze() for band in bands]
datasets = [ds.rename_vars({list(ds.data_vars)[0]: name}) for ds, name in zip(datasets, bands)]

combined = xr.merge(datasets)
combined

In [ ]:
count = xr.open_dataset(f'{basepath}_COUNT.tif').squeeze().to_array().squeeze()
count.plot.imshow()

In [ ]:
combined.red.plot.imshow(robust=True)